# Introduction

***Notebook in progress***

## Imports

In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)

# Helper Functions

Creating a function that randomly rolls the LCR die `num_dice` number of times and returns the total roll as a list.

In [6]:
def roll_LCR(num_dice):
    output = []
    for i in range(num_dice):
        roll = np.random.choice(["L", "C", "R", "O", "O", "O"])
        output.append(roll)
    return output

In [7]:
# Example
roll_LCR(8)

['L', 'C', 'O', 'C', 'O', 'L', 'C', 'O']